In [3]:
# import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import pandas as pd
import copy
import time

from numpy import *
from pandas import *
from google.colab import files, drive
from os import listdir

# 데이터 파일 불러옴

In [ ]:
df = pd.read_excel('데이터 파일 이름')

In [602]:
upload = files.upload()

Saving data_all.xlsx to data_all.xlsx


In [603]:
df = pd.read_excel('data_all.xlsx')
df

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,x_8,y_8,z_8,x_9,y_9,z_9,x_10,y_10,z_10,x_11,y_11,z_11,x_12,y_12,z_12,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적,y,image
0,0.802822,0.683172,-0.000059,0.796290,0.661867,-0.146656,0.757204,0.551149,-0.192669,0.706601,0.448169,-0.197359,0.655755,0.354045,-0.204960,0.790932,0.368932,-0.129292,0.698669,0.313862,-0.172512,0.632303,0.337803,-0.203215,0.582439,0.375781,-0.222803,0.767720,0.376558,-0.066249,0.674490,0.323270,-0.119961,0.611638,0.368772,-0.157459,0.563561,0.424743,-0.177124,0.747185,0.405188,-0.011417,0.671273,0.403154,-0.081756,0.651415,0.484210,-0.115277,0.636442,0.549586,-0.129835,0,0,0,0,0,1,correct18.MP4_000003.040.jpg
1,0.802112,0.672191,-0.000017,0.795411,0.642390,-0.143206,0.759011,0.525775,-0.185622,0.707753,0.423363,-0.191975,0.659722,0.332323,-0.206871,0.805196,0.353425,-0.120753,0.716508,0.304782,-0.167444,0.651719,0.327338,-0.198758,0.602915,0.368467,-0.217108,0.774002,0.358659,-0.055587,0.680201,0.312513,-0.119463,0.617398,0.358546,-0.161553,0.567457,0.416868,-0.181146,0.745328,0.382709,-0.000876,0.664769,0.395086,-0.077951,0.645512,0.484567,-0.118999,0.630237,0.551435,-0.134810,0,0,0,0,0,1,correct18.MP4_000003.160.jpg
2,0.834878,0.630622,-0.000036,0.832766,0.562037,-0.129846,0.795560,0.451895,-0.171299,0.735935,0.353962,-0.183522,0.687844,0.256539,-0.208985,0.823451,0.278121,-0.086189,0.737856,0.238672,-0.133510,0.679090,0.265810,-0.178718,0.640781,0.307479,-0.210603,0.787495,0.296410,-0.029600,0.700509,0.257458,-0.090715,0.640829,0.308430,-0.147433,0.605954,0.356631,-0.180794,0.757250,0.327631,0.016060,0.680012,0.327506,-0.052788,0.646495,0.405078,-0.102559,0.631131,0.453318,-0.129861,0,0,0,0,0,1,correct18.MP4_000003.640.jpg
3,0.840423,0.611011,-0.000028,0.835926,0.539953,-0.117908,0.799676,0.425365,-0.148123,0.748405,0.330711,-0.154980,0.702745,0.246166,-0.177059,0.816756,0.283106,-0.073382,0.738299,0.233189,-0.122390,0.679344,0.252439,-0.170702,0.635466,0.291793,-0.204810,0.782272,0.301644,-0.018568,0.701290,0.262287,-0.068624,0.643236,0.303859,-0.120193,0.605661,0.355507,-0.151100,0.752858,0.330315,0.024647,0.682568,0.320169,-0.040536,0.645990,0.383178,-0.094835,0.622334,0.434460,-0.124643,0,0,0,0,0,1,correct18.MP4_000003.761.jpg
4,0.846491,0.607810,-0.000035,0.832838,0.530135,-0.118343,0.795919,0.423277,-0.151772,0.743653,0.332124,-0.160703,0.700847,0.235392,-0.184964,0.826159,0.247272,-0.073614,0.737623,0.216855,-0.118516,0.674505,0.248014,-0.160364,0.632795,0.280961,-0.187706,0.794500,0.274452,-0.015906,0.707748,0.258341,-0.073416,0.660718,0.329992,-0.122204,0.640665,0.384702,-0.140170,0.766458,0.309036,0.030765,0.697225,0.319558,-0.036545,0.681332,0.406327,-0.077488,0.680488,0.460202,-0.090807,0,0,0,0,0,1,correct18.MP4_000003.880.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,-0.000051,0.683226,0.808627,-0.066687,0.691961,0.666358,-0.079175,0.706048,0.554577,-0.080995,0.723846,0.447428,-0.080970,0.687060,0.600681,0.005907,0.703182,0.481403,0.028084,0.724452,0.415654,0.039631,0.746156,0.370086,0.050197,0.694650,0.627415,0.051697,0.701560,0.514276,0.090203,0.717508,0.461997,0.105240,0.733055,0.419804,0.115534,0.701844,0.655029,0.089303,0.708349,0.560268,0.130411,0.719038,0.512440,0.142770,0.729677,0.475407,0.150917,0,0,0,0,0,0,not_correct7.mp4_000004.574.jpg
2081,0.689466,0.911463,-0.000065,0.662194,0.809515,-0.053108,0.658066,0.686111,-0.058303,0.662225,0.588759,-0.051490,0.672364,0.500283,-0.043548,0.684964,0.597477,0.009075,0.699973,0.486054,0.027304,0.711765,0.429146,0.027077,0.720656,0.390802,0.025083,0.695864,0.605227,0.048374,0.710104,0.490839,0.079638,0.720213,0.433627,0.090777,0.731818,0.400537,0.096398,0.703909,0.623557,0

In [604]:
for i in range(17):
  df = df.drop(['z_{}'.format(i)], axis=1)

In [605]:
df = df.drop(['d_ratio',	'd_ratio_sq',	'sin dif',	'엄지 중지 내적',	'검지 중지 내적',	'y'], axis=1)

In [606]:
df

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,x_6,y_6,x_7,y_7,x_8,y_8,x_9,y_9,x_10,y_10,x_11,y_11,x_12,y_12,x_13,y_13,x_14,y_14,x_15,y_15,x_16,y_16,image
0,0.802822,0.683172,0.796290,0.661867,0.757204,0.551149,0.706601,0.448169,0.655755,0.354045,0.790932,0.368932,0.698669,0.313862,0.632303,0.337803,0.582439,0.375781,0.767720,0.376558,0.674490,0.323270,0.611638,0.368772,0.563561,0.424743,0.747185,0.405188,0.671273,0.403154,0.651415,0.484210,0.636442,0.549586,correct18.MP4_000003.040.jpg
1,0.802112,0.672191,0.795411,0.642390,0.759011,0.525775,0.707753,0.423363,0.659722,0.332323,0.805196,0.353425,0.716508,0.304782,0.651719,0.327338,0.602915,0.368467,0.774002,0.358659,0.680201,0.312513,0.617398,0.358546,0.567457,0.416868,0.745328,0.382709,0.664769,0.395086,0.645512,0.484567,0.630237,0.551435,correct18.MP4_000003.160.jpg
2,0.834878,0.630622,0.832766,0.562037,0.795560,0.451895,0.735935,0.353962,0.687844,0.256539,0.823451,0.278121,0.737856,0.238672,0.679090,0.265810,0.640781,0.307479,0.787495,0.296410,0.700509,0.257458,0.640829,0.308430,0.605954,0.356631,0.757250,0.327631,0.680012,0.327506,0.646495,0.405078,0.631131,0.453318,correct18.MP4_000003.640.jpg
3,0.840423,0.611011,0.835926,0.539953,0.799676,0.425365,0.748405,0.330711,0.702745,0.246166,0.816756,0.283106,0.738299,0.233189,0.679344,0.252439,0.635466,0.291793,0.782272,0.301644,0.701290,0.262287,0.643236,0.303859,0.605661,0.355507,0.752858,0.330315,0.682568,0.320169,0.645990,0.383178,0.622334,0.434460,correct18.MP4_000003.761.jpg
4,0.846491,0.607810,0.832838,0.530135,0.795919,0.423277,0.743653,0.332124,0.700847,0.235392,0.826159,0.247272,0.737623,0.216855,0.674505,0.248014,0.632795,0.280961,0.794500,0.274452,0.707748,0.258341,0.660718,0.329992,0.640665,0.384702,0.766458,0.309036,0.697225,0.319558,0.681332,0.406327,0.680488,0.460202,correct18.MP4_000003.880.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,0.683226,0.808627,0.691961,0.666358,0.706048,0.554577,0.723846,0.447428,0.687060,0.600681,0.703182,0.481403,0.724452,0.415654,0.746156,0.370086,0.694650,0.627415,0.701560,0.514276,0.717508,0.461997,0.733055,0.419804,0.701844,0.655029,0.708349,0.560268,0.719038,0.512440,0.729677,0.475407,not_correct7.mp4_000004.574.jpg
2081,0.689466,0.911463,0.662194,0.809515,0.658066,0.686111,0.662225,0.588759,0.672364,0.500283,0.684964,0.597477,0.699973,0.486054,0.711765,0.429146,0.720656,0.390802,0.695864,0.605227,0.710104,0.490839,0.720213,0.433627,0.731818,0.400537,0.703909,0.623557,0.715664,0.525182,0.722518,0.468309,0.728340,0.438733,not_correct7.mp4_000005.110.jpg
2082,0.748213,0.857295,0.711621,0.746825,0.703696,0.626139,0.706495,0.528605,0.715309,0.446488,0.718847,0.552104,0.683949,0.455952,0.641702,0.441638,0.609599,0.460535,0.722917,0.556032,0.679210,0.440653,0.624662,0.430447,0.581561,0.463786,0.727055,0.570289,0.682644,0.477938,0.628725,0.473683,0.588093,0.493850,not_correct7.mp4_000005.377.jpg
2083,0.725273,0.837049,0.669997,0.710730,0.658297,0.590605,0.659387,0.499487,0.683194,0.432289,0.674882,0.507352,0.646205,0.392054,0.607197,0.391531,0.580782,0.435100,0.696248,0.510634,0.670623,0.388990,0.624398,0.417362,0.590639,0.475444,0.717677,0.520330,0.697354,0.409094,0.654995,0.441637,0.621378,0.496652,not_correct7.mp4_000005.507.jpg


이 상태가 어플에서 새로운 인풋이 들어왔을때 만든 엑셀 파일

# 불러온 데이터 파일에 preprocessing 할 column 추가 및 정렬

In [607]:
def add_columns(df):
  df["d ratio"] = 0
  df["d ratio sq"] = 0
  df["sin dif"] = 0
  df["엄지 중지 내적"] = 0
  df["검지 중지 내적"] = 0
  df = df.sort_values(by=["image"], ascending=[True])
  # return df
  pass

In [608]:
add_columns(df)

In [609]:
df

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,x_6,y_6,x_7,y_7,x_8,y_8,x_9,y_9,x_10,y_10,x_11,y_11,x_12,y_12,x_13,y_13,x_14,y_14,x_15,y_15,x_16,y_16,image,d ratio,d ratio sq,sin dif,엄지 중지 내적,검지 중지 내적
0,0.802822,0.683172,0.796290,0.661867,0.757204,0.551149,0.706601,0.448169,0.655755,0.354045,0.790932,0.368932,0.698669,0.313862,0.632303,0.337803,0.582439,0.375781,0.767720,0.376558,0.674490,0.323270,0.611638,0.368772,0.563561,0.424743,0.747185,0.405188,0.671273,0.403154,0.651415,0.484210,0.636442,0.549586,correct18.MP4_000003.040.jpg,0,0,0,0,0
1,0.802112,0.672191,0.795411,0.642390,0.759011,0.525775,0.707753,0.423363,0.659722,0.332323,0.805196,0.353425,0.716508,0.304782,0.651719,0.327338,0.602915,0.368467,0.774002,0.358659,0.680201,0.312513,0.617398,0.358546,0.567457,0.416868,0.745328,0.382709,0.664769,0.395086,0.645512,0.484567,0.630237,0.551435,correct18.MP4_000003.160.jpg,0,0,0,0,0
2,0.834878,0.630622,0.832766,0.562037,0.795560,0.451895,0.735935,0.353962,0.687844,0.256539,0.823451,0.278121,0.737856,0.238672,0.679090,0.265810,0.640781,0.307479,0.787495,0.296410,0.700509,0.257458,0.640829,0.308430,0.605954,0.356631,0.757250,0.327631,0.680012,0.327506,0.646495,0.405078,0.631131,0.453318,correct18.MP4_000003.640.jpg,0,0,0,0,0
3,0.840423,0.611011,0.835926,0.539953,0.799676,0.425365,0.748405,0.330711,0.702745,0.246166,0.816756,0.283106,0.738299,0.233189,0.679344,0.252439,0.635466,0.291793,0.782272,0.301644,0.701290,0.262287,0.643236,0.303859,0.605661,0.355507,0.752858,0.330315,0.682568,0.320169,0.645990,0.383178,0.622334,0.434460,correct18.MP4_000003.761.jpg,0,0,0,0,0
4,0.846491,0.607810,0.832838,0.530135,0.795919,0.423277,0.743653,0.332124,0.700847,0.235392,0.826159,0.247272,0.737623,0.216855,0.674505,0.248014,0.632795,0.280961,0.794500,0.274452,0.707748,0.258341,0.660718,0.329992,0.640665,0.384702,0.766458,0.309036,0.697225,0.319558,0.681332,0.406327,0.680488,0.460202,correct18.MP4_000003.880.jpg,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,0.683226,0.808627,0.691961,0.666358,0.706048,0.554577,0.723846,0.447428,0.687060,0.600681,0.703182,0.481403,0.724452,0.415654,0.746156,0.370086,0.694650,0.627415,0.701560,0.514276,0.717508,0.461997,0.733055,0.419804,0.701844,0.655029,0.708349,0.560268,0.719038,0.512440,0.729677,0.475407,not_correct7.mp4_000004.574.jpg,0,0,0,0,0
2081,0.689466,0.911463,0.662194,0.809515,0.658066,0.686111,0.662225,0.588759,0.672364,0.500283,0.684964,0.597477,0.699973,0.486054,0.711765,0.429146,0.720656,0.390802,0.695864,0.605227,0.710104,0.490839,0.720213,0.433627,0.731818,0.400537,0.703909,0.623557,0.715664,0.525182,0.722518,0.468309,0.728340,0.438733,not_correct7.mp4_000005.110.jpg,0,0,0,0,0
2082,0.748213,0.857295,0.711621,0.746825,0.703696,0.626139,0.706495,0.528605,0.715309,0.446488,0.718847,0.552104,0.683949,0.455952,0.641702,0.441638,0.609599,0.460535,0.722917,0.556032,0.679210,0.440653,0.624662,0.430447,0.581561,0.463786,0.727055,0.570289,0.682644,0.477938,0.628725,0.473683,0.588093,0.493850,not_correct7.mp4_000005.377.jpg,0,0,0,0,0
2083,0.725273,0.837049,0.669997,0.710730,0.658297,0.590605,0.659387,0.499487,0.683194,0.432289,0.674882,0.507352,0.646205,0.392054,0.607197,0.391531,0.580782,0.435100,0.696248,0.510634,0.670623,0.388990,0.624398,0.417362,0.590639,0.475444,0.717677,0.520330,0.697354,0.409094,0.654995,0.441637,0.621378,0.496652,not_correct7.mp4_000005.507.jpg,0,0,0,0,0


# 전처리 함수들

In [610]:
data['x_2-1']+data['x_2-1']

0      -1.165564
1      -1.061365
2      -1.469798
3      -1.283725
4      -1.370378
          ...   
2080    0.453421
2081   -0.273912
2082   -0.349387
2083   -0.362967
2084    0.809569
Name: x_2-1, Length: 2085, dtype: float64

In [611]:
for n in range(5):
  print(data.iloc[n,2]+data.iloc[n,2])

-1.1655640220406018
-1.061364502516659
-1.469798468288078
-1.2837252016045286
-1.3703783188529626


In [612]:
# d ratio, d ratio sq 만들어 주는 함수

def d_ratio(df):
  
  # df.iloc[i, 34] = ((df.iloc[i, 24]-df.iloc[i, 32])**2+(df.iloc[i, 25]-df.iloc[i, 33])**2)/((df.iloc[i, 24]-df.iloc[i, 16])**2+(df.iloc[i, 25]-df.iloc[i, 17])**2)
  df['d ratio'] = ((df['x_12']-df['x_16'])**2 + (df['y_12']-df['y_16'])**2)**0.5 / ((df['x_8']-df['x_12'])**2 + (df['y_8']-df['y_12'])**2)**0.5   # x,y 만 반영한 d ratio
  df['d ratio sq'] = ((df['x_12']-df['x_16'])**2 + (df['y_12']-df['y_16'])**2) / ((df['x_8']-df['x_12'])**2 + (df['y_8']-df['y_12'])**2)   # x,y 만 반영한 d ratio 의 제곱

  # return df
  pass

In [613]:
d_ratio(df)

In [614]:
df

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,x_6,y_6,x_7,y_7,x_8,y_8,x_9,y_9,x_10,y_10,x_11,y_11,x_12,y_12,x_13,y_13,x_14,y_14,x_15,y_15,x_16,y_16,image,d ratio,d ratio sq,sin dif,엄지 중지 내적,검지 중지 내적
0,0.802822,0.683172,0.796290,0.661867,0.757204,0.551149,0.706601,0.448169,0.655755,0.354045,0.790932,0.368932,0.698669,0.313862,0.632303,0.337803,0.582439,0.375781,0.767720,0.376558,0.674490,0.323270,0.611638,0.368772,0.563561,0.424743,0.747185,0.405188,0.671273,0.403154,0.651415,0.484210,0.636442,0.549586,correct18.MP4_000003.040.jpg,2.754793,7.588885,0,0,0
1,0.802112,0.672191,0.795411,0.642390,0.759011,0.525775,0.707753,0.423363,0.659722,0.332323,0.805196,0.353425,0.716508,0.304782,0.651719,0.327338,0.602915,0.368467,0.774002,0.358659,0.680201,0.312513,0.617398,0.358546,0.567457,0.416868,0.745328,0.382709,0.664769,0.395086,0.645512,0.484567,0.630237,0.551435,correct18.MP4_000003.160.jpg,2.474844,6.124852,0,0,0
2,0.834878,0.630622,0.832766,0.562037,0.795560,0.451895,0.735935,0.353962,0.687844,0.256539,0.823451,0.278121,0.737856,0.238672,0.679090,0.265810,0.640781,0.307479,0.787495,0.296410,0.700509,0.257458,0.640829,0.308430,0.605954,0.356631,0.757250,0.327631,0.680012,0.327506,0.646495,0.405078,0.631131,0.453318,correct18.MP4_000003.640.jpg,1.658572,2.750861,0,0,0
3,0.840423,0.611011,0.835926,0.539953,0.799676,0.425365,0.748405,0.330711,0.702745,0.246166,0.816756,0.283106,0.738299,0.233189,0.679344,0.252439,0.635466,0.291793,0.782272,0.301644,0.701290,0.262287,0.643236,0.303859,0.605661,0.355507,0.752858,0.330315,0.682568,0.320169,0.645990,0.383178,0.622334,0.434460,correct18.MP4_000003.761.jpg,1.147183,1.316029,0,0,0
4,0.846491,0.607810,0.832838,0.530135,0.795919,0.423277,0.743653,0.332124,0.700847,0.235392,0.826159,0.247272,0.737623,0.216855,0.674505,0.248014,0.632795,0.280961,0.794500,0.274452,0.707748,0.258341,0.660718,0.329992,0.640665,0.384702,0.766458,0.309036,0.697225,0.319558,0.681332,0.406327,0.680488,0.460202,correct18.MP4_000003.880.jpg,0.820459,0.673153,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,0.683226,0.808627,0.691961,0.666358,0.706048,0.554577,0.723846,0.447428,0.687060,0.600681,0.703182,0.481403,0.724452,0.415654,0.746156,0.370086,0.694650,0.627415,0.701560,0.514276,0.717508,0.461997,0.733055,0.419804,0.701844,0.655029,0.708349,0.560268,0.719038,0.512440,0.729677,0.475407,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,0,0,0
2081,0.689466,0.911463,0.662194,0.809515,0.658066,0.686111,0.662225,0.588759,0.672364,0.500283,0.684964,0.597477,0.699973,0.486054,0.711765,0.429146,0.720656,0.390802,0.695864,0.605227,0.710104,0.490839,0.720213,0.433627,0.731818,0.400537,0.703909,0.623557,0.715664,0.525182,0.722518,0.468309,0.728340,0.438733,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,0,0,0
2082,0.748213,0.857295,0.711621,0.746825,0.703696,0.626139,0.706495,0.528605,0.715309,0.446488,0.718847,0.552104,0.683949,0.455952,0.641702,0.441638,0.609599,0.460535,0.722917,0.556032,0.679210,0.440653,0.624662,0.430447,0.581561,0.463786,0.727055,0.570289,0.682644,0.477938,0.628725,0.473683,0.588093,0.493850,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0,0,0
2083,0.725273,0.837049,0.669997,0.710730,0.658297,0.590605,0.659387,0.499487,0.683194,0.432289,0.674882,0.507352,0.646205,0.392054,0.607197,0.391531,0.580782,0.435100,0.696248,0.510634,0.670623,0.388990,0.624398,0.417362,0.590639,0.475444,0.717677,0.520330,0.697354,0.409094,0.654995,0.441637,0.621378,0.496652,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0,0,0


In [615]:
# 벡터화 시키는 함수

def vectorization(df):
  global df_vector
  df_vector = df.copy()

  column = []
  for i in range (0, 16):
    column.append('x_{1}-{0}'.format(i,i+1))
    column.append('y_{1}-{0}'.format(i,i+1))
  column.append('image')
  column.append('d_ratio')
  column.append('d_ratio_sq')
  column.append('sin dif')
  column.append('엄지 중지 내적')
  column.append('검지 중지 내적')
  
 
  for i in range(0,16):
    df_vector['x_{0}'.format(i)] = df['x_{0}'.format(i+1)]-df['x_{0}'.format(i)]
    df_vector['y_{0}'.format(i)] = df['y_{0}'.format(i+1)]-df['y_{0}'.format(i)]

  df_vector = df_vector.drop(['x_16', 'y_16'], axis=1)

  df_vector.columns = column

  df_vector = pd.DataFrame(df_vector)
  # return df_vector
  pass

In [616]:
vectorization(df)

In [617]:
df_vector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.006532,-0.021305,-0.039086,-0.110718,-0.050603,-0.102979,-0.050845,-0.094125,0.135177,0.014888,-0.092264,-0.055070,-0.066365,0.023941,-0.049864,0.037977,0.185281,0.000777,-0.093230,-0.053288,-0.062852,0.045502,-0.048077,0.055971,0.183624,-0.019555,-0.075912,-0.002035,-0.019859,0.081056,-0.014972,0.065376,correct18.MP4_000003.040.jpg,2.754793,7.588885,0,0,0
1,-0.006701,-0.029801,-0.036400,-0.116615,-0.051258,-0.102412,-0.048031,-0.091040,0.145474,0.021102,-0.088688,-0.048642,-0.064789,0.022556,-0.048803,0.041128,0.171086,-0.009808,-0.093800,-0.046145,-0.062803,0.046033,-0.049941,0.058322,0.177871,-0.034159,-0.080559,0.012377,-0.019258,0.089481,-0.015275,0.066868,correct18.MP4_000003.160.jpg,2.474844,6.124852,0,0,0
2,-0.002112,-0.068584,-0.037206,-0.110143,-0.059625,-0.097933,-0.048092,-0.097422,0.135607,0.021582,-0.085595,-0.039449,-0.058766,0.027138,-0.038309,0.041669,0.146714,-0.011070,-0.086986,-0.038951,-0.059679,0.050971,-0.034875,0.048201,0.151296,-0.028999,-0.077239,-0.000125,-0.033516,0.077572,-0.015364,0.048240,correct18.MP4_000003.640.jpg,1.658572,2.750861,0,0,0
3,-0.004497,-0.071059,-0.036250,-0.114588,-0.051271,-0.094653,-0.045660,-0.084545,0.114010,0.036940,-0.078456,-0.049917,-0.058955,0.019250,-0.043879,0.039354,0.146807,0.009851,-0.080983,-0.039357,-0.058054,0.041571,-0.037575,0.051649,0.147197,-0.025192,-0.070289,-0.010146,-0.036578,0.063009,-0.023657,0.051282,correct18.MP4_000003.761.jpg,1.147183,1.316029,0,0,0
4,-0.013653,-0.077675,-0.036919,-0.106858,-0.052266,-0.091153,-0.042806,-0.096732,0.125312,0.011879,-0.088536,-0.030417,-0.063118,0.031159,-0.041710,0.032947,0.161705,-0.006510,-0.086752,-0.016110,-0.047030,0.071651,-0.020052,0.054709,0.125793,-0.075666,-0.069233,0.010522,-0.015893,0.086769,-0.000844,0.053875,correct18.MP4_000003.880.jpg,0.820459,0.673153,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.000061,-0.117157,0.008735,-0.142270,0.014087,-0.111780,0.017798,-0.107149,-0.036786,0.153253,0.016122,-0.119277,0.021270,-0.065749,0.021704,-0.045569,-0.051506,0.257329,0.006911,-0.113139,0.015947,-0.052279,0.015547,-0.042193,-0.031210,0.235225,0.006505,-0.094761,0.010689,-0.047828,0.010639,-0.037033,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,0,0,0
2081,-0.027271,-0.101948,-0.004128,-0.123403,0.004159,-0.097352,0.010139,-0.088476,0.012600,0.097194,0.015009,-0.111424,0.011793,-0.056908,0.008890,-0.038344,-0.024792,0.214425,0.014240,-0.114387,0.010109,-0.057212,0.011606,-0.033091,-0.027910,0.223021,0.011755,-0.098375,0.006854,-0.056873,0.005822,-0.029576,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,0,0,0
2082,-0.036592,-0.110470,-0.007924,-0.120686,0.002799,-0.097534,0.008814,-0.082118,0.003538,0.105616,-0.034898,-0.096151,-0.042246,-0.014314,-0.032104,0.018897,0.113319,0.095498,-0.043707,-0.115379,-0.054549,-0.010206,-0.043101,0.033339,0.145494,0.106503,-0.044411,-0.092351,-0.053920,-0.004256,-0.040632,0.020167,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0,0,0
2083,-0.055276,-0.126319,-0.011701,-0.120125,0.001091,-0.091118,0.023806,-0.067198,-0.008312,0.075063,-0.028676,-0.115298,-0.039009,-0.000523,-0.026415,0.043570,0.115466,0.075533,-0.025625,-0.121644,-0.046225,0.028373,-0.033758,0.058081,0.127038,0.044887,-0.020323,-0.111236,-0.042359,0.032543,-0.033617,0.055014,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0,0,0


In [618]:
# vector size 구하는 함수

def vectorsize():
  global df_size
  df_size = df_vector.copy()

  column_size =[]
  for i in range(0,16):
    column_size.append('size_{0}-{1}'.format(i+1,i))

  for i in range(0,16):
    df_size['x_{0}-{1}'.format(i+1, i)] = (df_vector['x_{0}-{1}'.format(i+1, i)]**2+df_vector['y_{0}-{1}'.format(i+1, i)]**2)**0.5
    df_size = df_size.drop(['y_{0}-{1}'.format(i+1, i)], axis=1)
  
  df_size = df_size.drop(['image',	'd_ratio',	'd_ratio_sq',	'sin dif','엄지 중지 내적',	'검지 중지 내적'], axis=1)
  df_size.columns = column_size
  pass

In [619]:
vectorsize()

In [620]:
df_size

,size_1-0,size_2-1,size_3-2,size_4-3,size_5-4,size_6-5,size_7-6,size_8-7,size_9-8,size_10-9,size_11-10,size_12-11,size_13-12,size_14-13,size_15-14,size_16-15
0,0.022284,0.117414,0.114741,0.106980,0.135994,0.107449,0.070552,0.062680,0.185282,0.107384,0.077594,0.073784,0.184662,0.075939,0.083454,0.067068
1,0.030545,0.122164,0.114523,0.102933,0.146996,0.101151,0.068603,0.063823,0.171367,0.104537,0.077867,0.076783,0.181122,0.081504,0.091530,0.068591
2,0.068617,0.116257,0.114656,0.108646,0.137314,0.094248,0.064729,0.056603,0.147131,0.095309,0.078484,0.059495,0.154050,0.077239,0.084503,0.050627
3,0.071201,0.120185,0.107647,0.096087,0.119845,0.092990,0.062018,0.058941,0.147137,0.090040,0.071403,0.063871,0.149337,0.071018,0.072857,0.056476
4,0.078865,0.113056,0.105074,0.105780,0.125874,0.093616,0.070390,0.053153,0.161836,0.088235,0.085707,0.058268,0.146796,0.070028,0.088213,0.053882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.117157,0.142538,0.112665,0.108617,0.157606,0.120362,0.069104,0.050473,0.262433,0.113350,0.054657,0.044966,0.237286,0.094984,0.049008,0.038531
2081,0.105533,0.123472,0.097441,0.089055,0.098007,0.112430,0.058117,0.039361,0.215853,0.115270,0.058098,0.035067,0.224760,0.099075,0.057284,0.030144
2082,0.116372,0.120946,0.097574,0.082589,0.105675,0.102288,0.044606,0.037252,0.148192,0.123380,0.055495,0.054490,0.180309,0.102474,0.054087,0.045361
2083,0.137883,0.120693,0.091125,0.071291,0.075522,0.118811,0.039012,0.050952,0.137977,0.124314,0.054238,0.067179,0.134734,0.113077,0.053417,0.064472


In [621]:
# vector size를 통해 단위벡터화 시키는 함수

def unitvector():
  global df_unitvector
  df_unitvector = df_vector.copy()

  for i in range(0,16):
    df_unitvector['x_{0}-{1}'.format(i+1, i)] = df_vector['x_{0}-{1}'.format(i+1, i)]/df_size['size_{0}-{1}'.format(i+1, i)]
    df_unitvector['y_{0}-{1}'.format(i+1, i)] = df_vector['y_{0}-{1}'.format(i+1, i)]/df_size['size_{0}-{1}'.format(i+1, i)]
    
  pass

In [622]:
unitvector()
df_unitvector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.293104,-0.956081,-0.332892,-0.942965,-0.441023,-0.897496,-0.475278,-0.879836,0.993990,0.109472,-0.858674,-0.512522,-0.940663,0.339343,-0.795543,0.605897,0.999991,0.004195,-0.868186,-0.496240,-0.810013,0.586412,-0.651581,0.758579,0.994377,-0.105895,-0.999641,-0.026792,-0.237961,0.971275,-0.223240,0.974763,correct18.MP4_000003.040.jpg,2.754793,7.588885,0,0,0
1,-0.219381,-0.975639,-0.297959,-0.954579,-0.447577,-0.894246,-0.466619,-0.884458,0.989643,0.143554,-0.876782,-0.480888,-0.944404,0.328786,-0.764673,0.644418,0.998361,-0.057234,-0.897297,-0.441428,-0.806543,0.591176,-0.650422,0.759573,0.982055,-0.188597,-0.988403,0.151853,-0.210399,0.977616,-0.222701,0.974887,correct18.MP4_000003.160.jpg,2.474844,6.124852,0,0,0
2,-0.030776,-0.999526,-0.320033,-0.947407,-0.520031,-0.854147,-0.442647,-0.896696,0.987571,0.157172,-0.908189,-0.418561,-0.907872,0.419247,-0.676801,0.736166,0.997166,-0.075237,-0.912676,-0.408684,-0.760404,0.649450,-0.586183,0.810178,0.982122,-0.188247,-0.999999,-0.001622,-0.396630,0.917979,-0.303469,0.952841,correct18.MP4_000003.640.jpg,1.658572,2.750861,0,0,0
3,-0.063163,-0.998003,-0.301617,-0.953429,-0.476286,-0.879291,-0.475193,-0.879882,0.951312,0.308228,-0.843708,-0.536802,-0.950607,0.310396,-0.744451,0.667677,0.997756,0.066954,-0.899411,-0.437104,-0.813042,0.582205,-0.588298,0.808644,0.985668,-0.168695,-0.989742,-0.142869,-0.502053,0.864837,-0.418885,0.908039,correct18.MP4_000003.761.jpg,1.147183,1.316029,0,0,0
4,-0.173120,-0.984901,-0.326555,-0.945178,-0.497417,-0.867511,-0.404672,-0.914462,0.995537,0.094375,-0.945745,-0.324911,-0.896686,0.442667,-0.784714,0.619858,0.999191,-0.040225,-0.983191,-0.182582,-0.548735,0.835996,-0.344137,0.938920,0.856920,-0.515449,-0.988647,0.150260,-0.180166,0.983636,-0.015670,0.999877,correct18.MP4_000003.880.jpg,0.820459,0.673153,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.000522,-1.000000,0.061284,-0.998120,0.125035,-0.992152,0.163855,-0.986484,-0.233404,0.972380,0.133946,-0.990989,0.307790,-0.951454,0.430008,-0.902825,-0.196262,0.980551,0.060968,-0.998140,0.291771,-0.956488,0.345749,-0.938327,-0.131531,0.991312,0.068484,-0.997652,0.218110,-0.975924,0.276109,-0.961126,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,0,0,0
2081,-0.258417,-0.966033,-0.033435,-0.999441,0.042680,-0.999089,0.113852,-0.993498,0.128558,0.991702,0.133499,-0.991049,0.202913,-0.979197,0.225865,-0.974159,-0.114854,0.993382,0.123533,-0.992340,0.174002,-0.984745,0.330955,-0.943647,-0.124175,0.992260,0.118646,-0.992937,0.119649,-0.992816,0.193145,-0.981170,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,0,0,0
2082,-0.314442,-0.949277,-0.065520,-0.997851,0.028684,-0.999589,0.106715,-0.994290,0.033481,0.999439,-0.341173,-0.940000,-0.947109,-0.320911,-0.861791,0.507263,0.764675,0.644417,-0.354247,-0.935152,-0.982943,-0.183911,-0.790987,0.611833,0.806913,0.590670,-0.433385,-0.901209,-0.996900,-0.078680,-0.895738,0.444582,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0,0,0
2083,-0.400889,-0.916127,-0.096946,-0.995290,0.011969,-0.999928,0.333933,-0.942597,-0.110061,0.993925,-0.241361,-0.970435,-0.999910,-0.013416,-0.518431,0.855119,0.836850,0.547432,-0.206134,-0.978524,-0.852265,0.523110,-0.502510,0.864571,0.942874,0.333149,-0.179727,-0.983717,-0.792995,0.609228,-0.521417,0.853302,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0,0,0


In [623]:
for n in range(3):
  for i in range(16):
    print(df_unitvector.iloc[n,2*i]**2+df_unitvector.iloc[n,2*i+1]**2)

1.0000000000000002
0.9999999999999999
0.9999999999999998
0.9999999999999998
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999998
1.0
1.0
0.9999999999999998
1.0000000000000002
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0


In [624]:
# 중지 약지의 sin 값 차이 데이터 만드는 함수

def sin_dif():
  global df_vector
  df_vector['cos 7-6, 11-10'] =0
  df_vector['sin 7-6, 11-10'] = 0
  df_vector['cos 11-10, 15-14'] = 0
  df_vector['sin 11-10, 15-14'] = 0

  df_vector['cos 7-6, 11-10'] = df_unitvector['x_7-6']*df_unitvector['x_11-10'] + df_unitvector['y_7-6']*df_unitvector['y_11-10']
  df_vector['sin 7-6, 11-10'] = df_unitvector['y_7-6']*df_unitvector['x_11-10'] - df_unitvector['x_7-6']*df_unitvector['y_11-10']
  df_vector['cos 11-10, 15-14'] = df_unitvector['x_11-10']*df_unitvector['x_15-14'] + df_unitvector['y_11-10']*df_unitvector['y_15-14']
  df_vector['sin 11-10, 15-14'] = df_unitvector['y_11-10']*df_unitvector['x_15-14'] - df_unitvector['x_11-10']*df_unitvector['y_15-14']

  df_vector['sin dif'] = df_vector['sin 7-6, 11-10']*df_vector['cos 11-10, 15-14'] - df_vector['cos 7-6, 11-10']*df_vector['sin 11-10, 15-14']

  df_vector = df_vector.drop(['cos 7-6, 11-10',	'sin 7-6, 11-10',	'cos 11-10, 15-14',	'sin 11-10, 15-14'], axis=1)

  pass

In [625]:
sin_dif()
df_vector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.006532,-0.021305,-0.039086,-0.110718,-0.050603,-0.102979,-0.050845,-0.094125,0.135177,0.014888,-0.092264,-0.055070,-0.066365,0.023941,-0.049864,0.037977,0.185281,0.000777,-0.093230,-0.053288,-0.062852,0.045502,-0.048077,0.055971,0.183624,-0.019555,-0.075912,-0.002035,-0.019859,0.081056,-0.014972,0.065376,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,0,0
1,-0.006701,-0.029801,-0.036400,-0.116615,-0.051258,-0.102412,-0.048031,-0.091040,0.145474,0.021102,-0.088688,-0.048642,-0.064789,0.022556,-0.048803,0.041128,0.171086,-0.009808,-0.093800,-0.046145,-0.062803,0.046033,-0.049941,0.058322,0.177871,-0.034159,-0.080559,0.012377,-0.019258,0.089481,-0.015275,0.066868,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,0,0
2,-0.002112,-0.068584,-0.037206,-0.110143,-0.059625,-0.097933,-0.048092,-0.097422,0.135607,0.021582,-0.085595,-0.039449,-0.058766,0.027138,-0.038309,0.041669,0.146714,-0.011070,-0.086986,-0.038951,-0.059679,0.050971,-0.034875,0.048201,0.151296,-0.028999,-0.077239,-0.000125,-0.033516,0.077572,-0.015364,0.048240,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,0,0
3,-0.004497,-0.071059,-0.036250,-0.114588,-0.051271,-0.094653,-0.045660,-0.084545,0.114010,0.036940,-0.078456,-0.049917,-0.058955,0.019250,-0.043879,0.039354,0.146807,0.009851,-0.080983,-0.039357,-0.058054,0.041571,-0.037575,0.051649,0.147197,-0.025192,-0.070289,-0.010146,-0.036578,0.063009,-0.023657,0.051282,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,0,0
4,-0.013653,-0.077675,-0.036919,-0.106858,-0.052266,-0.091153,-0.042806,-0.096732,0.125312,0.011879,-0.088536,-0.030417,-0.063118,0.031159,-0.041710,0.032947,0.161705,-0.006510,-0.086752,-0.016110,-0.047030,0.071651,-0.020052,0.054709,0.125793,-0.075666,-0.069233,0.010522,-0.015893,0.086769,-0.000844,0.053875,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.000061,-0.117157,0.008735,-0.142270,0.014087,-0.111780,0.017798,-0.107149,-0.036786,0.153253,0.016122,-0.119277,0.021270,-0.065749,0.021704,-0.045569,-0.051506,0.257329,0.006911,-0.113139,0.015947,-0.052279,0.015547,-0.042193,-0.031210,0.235225,0.006505,-0.094761,0.010689,-0.047828,0.010639,-0.037033,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0,0
2081,-0.027271,-0.101948,-0.004128,-0.123403,0.004159,-0.097352,0.010139,-0.088476,0.012600,0.097194,0.015009,-0.111424,0.011793,-0.056908,0.008890,-0.038344,-0.024792,0.214425,0.014240,-0.114387,0.010109,-0.057212,0.011606,-0.033091,-0.027910,0.223021,0.011755,-0.098375,0.006854,-0.056873,0.005822,-0.029576,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0,0
2082,-0.036592,-0.110470,-0.007924,-0.120686,0.002799,-0.097534,0.008814,-0.082118,0.003538,0.105616,-0.034898,-0.096151,-0.042246,-0.014314,-0.032104,0.018897,0.113319,0.095498,-0.043707,-0.115379,-0.054549,-0.010206,-0.043101,0.033339,0.145494,0.106503,-0.044411,-0.092351,-0.053920,-0.004256,-0.040632,0.020167,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,0,0
2083,-0.055276,-0.126319,-0.011701,-0.120125,0.001091,-0.091118,0.023806,-0.067198,-0.008312,0.075063,-0.028676,-0.115298,-0.039009,-0.000523,-0.026415,0.043570,0.115466,0.075533,-0.025625,-0.121644,-0.046225,0.028373,-0.033758,0.058081,0.127038,0.044887,-0.020323,-0.111236,-0.042359,0.032543,-0.033617,0.055014,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,0,0


In [626]:
# 엄지 중지 첫 마디 내적값 만드는 함수

def dotproduct_tm():
  global df_unitvector, df_vector
  
  df_vector['엄지 중지 내적'] = df_unitvector['x_4-3']*df_unitvector['x_12-11'] + df_unitvector['y_4-3']*df_unitvector['y_12-11']
  # df_vector['검지 중지 내적'] = df_unitvector['x_8-7']*df_unitvector['x_12-11'] + df_unitvector['y_8-7']*df_unitvector['y_12-11']

  pass

In [627]:
dotproduct_tm()
df_vector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.006532,-0.021305,-0.039086,-0.110718,-0.050603,-0.102979,-0.050845,-0.094125,0.135177,0.014888,-0.092264,-0.055070,-0.066365,0.023941,-0.049864,0.037977,0.185281,0.000777,-0.093230,-0.053288,-0.062852,0.045502,-0.048077,0.055971,0.183624,-0.019555,-0.075912,-0.002035,-0.019859,0.081056,-0.014972,0.065376,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0
1,-0.006701,-0.029801,-0.036400,-0.116615,-0.051258,-0.102412,-0.048031,-0.091040,0.145474,0.021102,-0.088688,-0.048642,-0.064789,0.022556,-0.048803,0.041128,0.171086,-0.009808,-0.093800,-0.046145,-0.062803,0.046033,-0.049941,0.058322,0.177871,-0.034159,-0.080559,0.012377,-0.019258,0.089481,-0.015275,0.066868,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0
2,-0.002112,-0.068584,-0.037206,-0.110143,-0.059625,-0.097933,-0.048092,-0.097422,0.135607,0.021582,-0.085595,-0.039449,-0.058766,0.027138,-0.038309,0.041669,0.146714,-0.011070,-0.086986,-0.038951,-0.059679,0.050971,-0.034875,0.048201,0.151296,-0.028999,-0.077239,-0.000125,-0.033516,0.077572,-0.015364,0.048240,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0
3,-0.004497,-0.071059,-0.036250,-0.114588,-0.051271,-0.094653,-0.045660,-0.084545,0.114010,0.036940,-0.078456,-0.049917,-0.058955,0.019250,-0.043879,0.039354,0.146807,0.009851,-0.080983,-0.039357,-0.058054,0.041571,-0.037575,0.051649,0.147197,-0.025192,-0.070289,-0.010146,-0.036578,0.063009,-0.023657,0.051282,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0
4,-0.013653,-0.077675,-0.036919,-0.106858,-0.052266,-0.091153,-0.042806,-0.096732,0.125312,0.011879,-0.088536,-0.030417,-0.063118,0.031159,-0.041710,0.032947,0.161705,-0.006510,-0.086752,-0.016110,-0.047030,0.071651,-0.020052,0.054709,0.125793,-0.075666,-0.069233,0.010522,-0.015893,0.086769,-0.000844,0.053875,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.000061,-0.117157,0.008735,-0.142270,0.014087,-0.111780,0.017798,-0.107149,-0.036786,0.153253,0.016122,-0.119277,0.021270,-0.065749,0.021704,-0.045569,-0.051506,0.257329,0.006911,-0.113139,0.015947,-0.052279,0.015547,-0.042193,-0.031210,0.235225,0.006505,-0.094761,0.010689,-0.047828,0.010639,-0.037033,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0
2081,-0.027271,-0.101948,-0.004128,-0.123403,0.004159,-0.097352,0.010139,-0.088476,0.012600,0.097194,0.015009,-0.111424,0.011793,-0.056908,0.008890,-0.038344,-0.024792,0.214425,0.014240,-0.114387,0.010109,-0.057212,0.011606,-0.033091,-0.027910,0.223021,0.011755,-0.098375,0.006854,-0.056873,0.005822,-0.029576,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0
2082,-0.036592,-0.110470,-0.007924,-0.120686,0.002799,-0.097534,0.008814,-0.082118,0.003538,0.105616,-0.034898,-0.096151,-0.042246,-0.014314,-0.032104,0.018897,0.113319,0.095498,-0.043707,-0.115379,-0.054549,-0.010206,-0.043101,0.033339,0.145494,0.106503,-0.044411,-0.092351,-0.053920,-0.004256,-0.040632,0.020167,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0
2083,-0.055276,-0.126319,-0.011701,-0.120125,0.001091,-0.091118,0.023806,-0.067198,-0.008312,0.075063,-0.028676,-0.115298,-0.039009,-0.000523,-0.026415,0.043570,0.115466,0.075533,-0.025625,-0.121644,-0.046225,0.028373,-0.033758,0.058081,0.127038,0.044887,-0.020323,-0.111236,-0.042359,0.032543,-0.033617,0.055014,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0


In [628]:
# 검지 중지 첫 마디 내적값 만드는 함수

def dotproduct_im():
  global df_unitvector, df_vector

  df_vector['검지 중지 내적'] = df_unitvector['x_8-7']*df_unitvector['x_12-11'] + df_unitvector['y_8-7']*df_unitvector['y_12-11']

  pass

In [629]:
dotproduct_im()
df_vector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.006532,-0.021305,-0.039086,-0.110718,-0.050603,-0.102979,-0.050845,-0.094125,0.135177,0.014888,-0.092264,-0.055070,-0.066365,0.023941,-0.049864,0.037977,0.185281,0.000777,-0.093230,-0.053288,-0.062852,0.045502,-0.048077,0.055971,0.183624,-0.019555,-0.075912,-0.002035,-0.019859,0.081056,-0.014972,0.065376,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0.977981
1,-0.006701,-0.029801,-0.036400,-0.116615,-0.051258,-0.102412,-0.048031,-0.091040,0.145474,0.021102,-0.088688,-0.048642,-0.064789,0.022556,-0.048803,0.041128,0.171086,-0.009808,-0.093800,-0.046145,-0.062803,0.046033,-0.049941,0.058322,0.177871,-0.034159,-0.080559,0.012377,-0.019258,0.089481,-0.015275,0.066868,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0.986843
2,-0.002112,-0.068584,-0.037206,-0.110143,-0.059625,-0.097933,-0.048092,-0.097422,0.135607,0.021582,-0.085595,-0.039449,-0.058766,0.027138,-0.038309,0.041669,0.146714,-0.011070,-0.086986,-0.038951,-0.059679,0.050971,-0.034875,0.048201,0.151296,-0.028999,-0.077239,-0.000125,-0.033516,0.077572,-0.015364,0.048240,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0.993155
3,-0.004497,-0.071059,-0.036250,-0.114588,-0.051271,-0.094653,-0.045660,-0.084545,0.114010,0.036940,-0.078456,-0.049917,-0.058955,0.019250,-0.043879,0.039354,0.146807,0.009851,-0.080983,-0.039357,-0.058054,0.041571,-0.037575,0.051649,0.147197,-0.025192,-0.070289,-0.010146,-0.036578,0.063009,-0.023657,0.051282,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0.977872
4,-0.013653,-0.077675,-0.036919,-0.106858,-0.052266,-0.091153,-0.042806,-0.096732,0.125312,0.011879,-0.088536,-0.030417,-0.063118,0.031159,-0.041710,0.032947,0.161705,-0.006510,-0.086752,-0.016110,-0.047030,0.071651,-0.020052,0.054709,0.125793,-0.075666,-0.069233,0.010522,-0.015893,0.086769,-0.000844,0.053875,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0.852046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.000061,-0.117157,0.008735,-0.142270,0.014087,-0.111780,0.017798,-0.107149,-0.036786,0.153253,0.016122,-0.119277,0.021270,-0.065749,0.021704,-0.045569,-0.051506,0.257329,0.006911,-0.113139,0.015947,-0.052279,0.015547,-0.042193,-0.031210,0.235225,0.006505,-0.094761,0.010689,-0.047828,0.010639,-0.037033,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0.995820
2081,-0.027271,-0.101948,-0.004128,-0.123403,0.004159,-0.097352,0.010139,-0.088476,0.012600,0.097194,0.015009,-0.111424,0.011793,-0.056908,0.008890,-0.038344,-0.024792,0.214425,0.014240,-0.114387,0.010109,-0.057212,0.011606,-0.033091,-0.027910,0.223021,0.011755,-0.098375,0.006854,-0.056873,0.005822,-0.029576,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0.994013
2082,-0.036592,-0.110470,-0.007924,-0.120686,0.002799,-0.097534,0.008814,-0.082118,0.003538,0.105616,-0.034898,-0.096151,-0.042246,-0.014314,-0.032104,0.018897,0.113319,0.095498,-0.043707,-0.115379,-0.054549,-0.010206,-0.043101,0.033339,0.145494,0.106503,-0.044411,-0.092351,-0.053920,-0.004256,-0.040632,0.020167,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0.992026
2083,-0.055276,-0.126319,-0.011701,-0.120125,0.001091,-0.091118,0.023806,-0.067198,-0.008312,0.075063,-0.028676,-0.115298,-0.039009,-0.000523,-0.026415,0.043570,0.115466,0.075533,-0.025625,-0.121644,-0.046225,0.028373,-0.033758,0.058081,0.127038,0.044887,-0.020323,-0.111236,-0.042359,0.032543,-0.033617,0.055014,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0.999829


In [630]:
# 16-15 크기 정규화 시키는 함수

def normed_1615():
  global df_vector, df_size

  for n in range(len(df)):
    for i in range(16):
      df_vector.iloc[n,2*i] = df_vector.iloc[n,2*i]/df_size.iloc[n, 15]
      df_vector.iloc[n,2*i+1] = df_vector.iloc[n,2*i+1]/df_size.iloc[n, 15]

  pass

In [631]:
normed_1615()
df_vector

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,-0.097386,-0.317665,-0.582782,-1.650816,-0.754504,-1.535439,-0.758110,-1.403414,2.015508,0.221975,-1.375666,-0.821102,-0.989515,0.356967,-0.743485,0.566249,2.762563,0.011589,-1.390067,-0.794538,-0.937136,0.678444,-0.716828,0.834541,2.737856,-0.291565,-1.131854,-0.030336,-0.296096,1.208562,-0.223240,0.974763,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0.977981
1,-0.097696,-0.434478,-0.530682,-1.700158,-0.747303,-1.493090,-0.700252,-1.327300,2.120896,0.307649,-1.292999,-0.709170,-0.944578,0.328847,-0.711517,0.599622,2.494309,-0.142994,-1.367539,-0.672766,-0.915622,0.671128,-0.728103,0.850291,2.593228,-0.498012,-1.174490,0.180443,-0.280763,1.304564,-0.222701,0.974887,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0.986843
2,-0.041711,-1.354691,-0.734899,-2.175554,-1.177719,-1.934394,-0.949919,-1.924305,2.678537,0.426290,-1.690685,-0.779194,-1.160758,0.536027,-0.756683,0.823056,2.897922,-0.218650,-1.718170,-0.769372,-1.178799,1.006795,-0.688852,0.952079,2.988419,-0.572803,-1.525631,-0.002475,-0.662021,1.532212,-0.303469,0.952841,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0.993155
3,-0.079632,-1.258210,-0.641863,-2.028964,-0.907833,-1.675988,-0.808480,-1.497007,2.018738,0.654078,-1.389198,-0.883864,-1.043890,0.340855,-0.776948,0.696822,2.599454,0.174436,-1.433933,-0.696877,-1.027934,0.736085,-0.665327,0.914524,2.606360,-0.446073,-1.244587,-0.179655,-0.647670,1.115677,-0.418885,0.908039,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0.977872
4,-0.253392,-1.441579,-0.685189,-1.983205,-0.970009,-1.691725,-0.794451,-1.795269,2.325695,0.220472,-1.643168,-0.564511,-1.171413,0.578292,-0.774101,0.611475,3.001122,-0.120818,-1.610056,-0.298993,-0.872849,1.329783,-0.372155,1.015364,2.334615,-1.404302,-1.284908,0.195288,-0.294960,1.610369,-0.015670,0.999877,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0.852046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,-0.001589,-3.040612,0.226711,-3.692383,0.365606,-2.901081,0.461906,-2.780889,-0.954719,3.977435,0.418422,-3.095655,0.552016,-1.706414,0.563291,-1.182662,-1.336750,6.678570,0.179356,-2.936343,0.413889,-1.356818,0.403498,-1.095053,-0.810018,6.104883,0.168824,-2.459373,0.277418,-1.241298,0.276109,-0.961126,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0.995820
2081,-0.904720,-3.382087,-0.136956,-4.093858,0.137966,-3.229617,0.336361,-2.935153,0.417988,3.224375,0.497927,-3.696444,0.391217,-1.887897,0.294931,-1.272042,-0.822453,7.113460,0.472395,-3.794752,0.335370,-1.897989,0.385008,-1.097767,-0.925893,7.398620,0.389961,-3.263557,0.227379,-1.886734,0.193145,-0.981170,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0.994013
2082,-0.806689,-2.435328,-0.174693,-2.660553,0.061701,-2.150159,0.194296,-1.810301,0.077999,2.328325,-0.769337,-2.119675,-0.931329,-0.315565,-0.707735,0.416584,2.498143,2.105267,-0.963533,-2.543564,-1.202535,-0.224997,-0.950166,0.734959,3.207447,2.347889,-0.979045,-2.035890,-1.188673,-0.093816,-0.895738,0.444582,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0.992026
2083,-0.857359,-1.959267,-0.181484,-1.863195,0.016917,-1.413289,0.369248,-1.042282,-0.128924,1.164268,-0.444784,-1.788333,-0.605044,-0.008118,-0.409710,0.675791,1.790943,1.171561,-0.397461,-1.886762,-0.716980,0.440074,-0.523609,0.900872,1.970419,0.696214,-0.315221,-1.725326,-0.657011,0.504756,-0.521417,0.853302,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0.999829


In [632]:
# 3-2 벡터를 기준으로 회전 정규화 시키는 함수

def rotate_32():
  global df_vector, df_unitvector, df_vector_final
  df_vector_final = df_vector.copy()

  for i in range(0,16):
    df_vector_final['x_{0}-{1}'.format(i+1, i)] = df_vector['x_{0}-{1}'.format(i+1, i)]*df_unitvector['x_3-2'] + df_vector['y_{0}-{1}'.format(i+1, i)]*df_unitvector['y_3-2']
    df_vector_final['y_{0}-{1}'.format(i+1, i)] = -df_vector['x_{0}-{1}'.format(i+1, i)]*df_unitvector['y_3-2'] + df_vector['y_{0}-{1}'.format(i+1, i)]*df_unitvector['x_3-2']

  pass    

In [633]:
rotate_32()
df_vector_final

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,0.328053,0.052694,1.738621,0.205004,1.710803,0.000000e+00,1.593903,-0.061462,-1.088107,1.711014,1.343636,-0.872529,0.116023,-1.045516,-0.180312,-0.917004,-1.228755,2.474277,1.326146,-0.897170,-0.195602,-1.140285,-0.432859,-1.011401,-0.945779,2.585801,0.526400,-1.002456,-0.954094,-0.798749,-0.776392,-0.630251,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0.977981
1,0.432257,0.107098,1.757880,0.286390,1.669664,0.000000e+00,1.500348,-0.032129,-1.224377,1.758906,1.212888,-0.838851,0.128701,-0.991869,-0.217751,-0.904648,-0.988522,2.294526,1.213696,-0.921801,-0.190343,-1.119172,-0.434487,-1.031673,-0.715323,2.541882,0.364314,-1.131044,-1.040938,-0.834963,-0.772113,-0.635486,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0.986843
2,1.178796,0.668854,2.240414,0.503644,2.264708,0.000000e+00,2.137628,0.189328,-1.757037,2.066180,1.544756,-1.038888,0.145784,-1.270209,-0.309512,-1.074334,-1.320251,2.588957,1.550659,-1.067472,-0.246939,-1.530433,-0.454991,-1.083492,-1.064814,2.850425,0.795489,-1.301826,-0.964463,-1.362262,-0.656054,-0.754714,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0.993155
3,1.144260,0.529248,2.089759,0.401983,1.906068,-1.110223e-16,1.701371,0.002114,-1.536620,1.463530,1.438829,-0.800537,0.197479,-1.080227,-0.242660,-1.015050,-1.391463,2.202594,1.295719,-0.928932,-0.157643,-1.254440,-0.487246,-1.020591,-0.849144,2.504206,0.750748,-1.008786,-0.672528,-1.100871,-0.598922,-0.800808,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0.977872
4,1.376628,0.497246,2.061278,0.392072,1.950090,1.110223e-16,1.952590,0.203803,-1.348104,1.907900,1.307060,-1.144669,0.081007,-1.303867,-0.145410,-0.975700,-1.388000,2.663604,1.060249,-1.248017,-0.719432,-1.418663,-0.695723,-0.827909,0.056969,2.723829,0.469721,-1.211811,-1.250295,-1.056907,-0.859611,-0.510950,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0.852046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,3.016551,-0.381759,3.691754,-0.236746,2.924027,0.000000e+00,2.816820,0.110573,-4.065595,-0.449908,3.123679,0.028073,1.762044,0.334322,1.243812,0.410997,-6.793299,-0.491204,2.935725,-0.189198,1.397921,0.240991,1.136911,0.263412,-6.158255,-0.040337,2.461181,-0.140009,1.266243,0.120035,0.988107,0.153768,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0.995820
2081,3.340392,-1.048243,4.084283,-0.311558,3.232563,-2.775558e-17,2.946834,0.210782,-3.203597,0.555224,3.714328,0.339708,1.902874,0.310285,1.283471,0.240372,-7.142081,-0.518100,3.811456,0.310004,1.910574,0.254058,1.113199,0.337804,-7.431396,-0.609275,3.277226,0.250317,1.894720,0.146645,0.988520,0.151092,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0.994013
2082,2.411187,-0.876213,2.654447,-0.250938,2.151044,6.938894e-18,1.815129,0.142288,-2.325130,0.144753,2.096735,-0.829822,0.288720,-0.939998,-0.436713,-0.695495,-2.032743,2.557503,2.514879,-1.036097,0.190411,-1.208494,-0.761911,-0.928693,-2.254920,3.273475,2.006969,-1.037040,0.059681,-1.190875,-0.470092,-0.882617,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0.992026
2083,1.948865,-0.880748,1.860890,-0.203772,1.413390,-3.469447e-18,1.046627,0.356746,-1.165728,-0.114979,1.782881,-0.466157,0.000875,-0.605097,-0.680646,-0.401592,-1.150040,1.804838,1.881870,-0.420016,-0.448624,-0.711661,-0.907075,-0.512789,-0.672580,1.978611,1.721430,-0.335849,-0.512584,-0.650922,-0.859482,-0.511166,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0.999829


In [635]:
df_vector_final = df_vector_final.sort_values(by=["image"], ascending=[True])

df_save = pd.DataFrame(df_vector_final)

with pd.ExcelWriter('preprocessed_0826_all.xlsx') as writer:
  df_save_1.to_excel(writer, sheet_name='data',index=False, header=True)
  
files.download('preprocessed_0826_all.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>